Run `pip install .` first to install all dependencies.

In [1]:
!pip install gcsfs==2022.10.0
!pip install wandb==0.13.4
!pip install fastparquet
!pip install numpy
!pip install pandas

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 138 kB 6.2 MB/s 
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2022.11.0
    Uninstalling fsspec-2022.11.0:
      Successfully uninstalled fsspec-2022.11.0
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.9 MB 4.0 MB/s 
     |████████████████████████████████| 168 kB 45.2 MB/s 
     |████████████████████████████████| 182 kB 72.0 MB/s 
     |████████████████████████████████| 62 kB 474 kB/s 
     |████████████████████████████████| 168 kB 64.7 MB/s 
     |████████████████████████████████| 166 kB 60.6 MB/s 
     |████████████████████████████████| 166 kB 46.8 MB/s 
     |████████████████████████████████| 162 kB 37.5 MB/s 
     |████████████████████████████████| 162 kB 67.8 MB/s 
     |████████████████████████████████| 158 kB 65.2 MB/s 
    

In [2]:
import gcsfs
import numpy as np
import pandas as pd

import wandb

from sklearn.preprocessing import MinMaxScaler


In [3]:
name = "classical_size_features_log_normalized"

# connect to weights and biases
run = wandb.init(project="thesis", job_type="dataset-creation", entity="fbv")

dataset = wandb.Artifact(name=name, type="preprocessed_data")


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 

··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [4]:
import google.auth
from google.colab import auth
# connect to google cloud storage
auth.authenticate_user()
credentials, _ = google.auth.default()
fs = gcsfs.GCSFileSystem(project="thesis", token=credentials)

In [5]:
# reduce number of imported cols due to memory issues
columns = [
    "QUOTE_DATETIME",
    "ROOT",
    "EXPIRATION",
    "STRK_PRC",
    "OPTION_TYPE",
    "TRADE_SIZE",
    "TRADE_PRICE",
    "BEST_BID",
    "BEST_ASK",
    "ask_ex",
    "bid_ex",
    "bid_size_ex",
    "ask_size_ex",
    "price_all_lead",
    "price_all_lag",
    "price_ex_lead",
    "price_ex_lag",
    "buy_sell",
]


In [6]:
train = pd.read_parquet(
    f"gs://thesis-bucket-option-trade-classification/data/preprocessed/train_set_extended_60.parquet",
    engine="fastparquet",
    columns=columns,
)
val = pd.read_parquet(
    f"gs://thesis-bucket-option-trade-classification/data/preprocessed/val_set_extended_20.parquet",
    engine="fastparquet",
    columns=columns,
)
test = pd.read_parquet(
    f"gs://thesis-bucket-option-trade-classification/data/preprocessed/test_set_extended_20.parquet",
    engine="fastparquet",
    columns=columns,
)


In [7]:
# oe_option_type = OrdinalEncoder(
#     unknown_value=-1, dtype=int, handle_unknown="use_encoded_value"
# )
# oe_root = OrdinalEncoder(
#     unknown_value=-1, dtype=int, handle_unknown="use_encoded_value"
# )
# oe_issue_type = OrdinalEncoder(
#     unknown_value=-1, dtype=int, handle_unknown="use_encoded_value"
# )

min_max_scaler = MinMaxScaler(feature_range=[-1, 1])


def transform(data: pd.DataFrame) -> pd.DataFrame:

    # # date features
    x = pd.DataFrame(data={"TRADE_PRICE": data["TRADE_PRICE"]}, index=data.index)

    # x["date_month_sin"] = np.sin(2 * np.pi * data["QUOTE_DATETIME"].dt.year / 12)
    # x["date_month_cos"] = np.cos(2 * np.pi * data["QUOTE_DATETIME"].dt.year / 12)

    # seconds_in_day = 24 * 60 * 60
    # seconds = (
    #     data["QUOTE_DATETIME"] - data["QUOTE_DATETIME"].dt.normalize()
    # ).dt.total_seconds()

    # x["date_time_sin"] = np.sin(2 * np.pi * seconds / seconds_in_day)
    # x["date_time_cos"] = np.cos(2 * np.pi * seconds / seconds_in_day)

    # option features
    # x["ttm"] = (
    #     data["EXPIRATION"].dt.to_period("M") - data["QUOTE_DATETIME"].dt.to_period("M")
    # ).apply(lambda x: x.n)
    # x[["myn", "day_vol"]] = data[["myn", "day_vol"]]
    # x["log_strk_prc"] = np.log1p(data["STRK_PRC"])

    # binarize
    # "bin_OPTION_TYPE", "bin_issue_type", "bin_ROOT",

    # size features
    x["bid_ask_size_ratio_ex"] = data["bid_size_ex"] / data["ask_size_ex"]
    x["rel_bid_size_ex"] = data["TRADE_SIZE"] / data["bid_size_ex"]
    x["rel_ask_size_ex"] = data["TRADE_SIZE"] / data["ask_size_ex"]
    # x[["TRADE_SIZE", "bid_size_ex", "ask_size_ex"]] = data[
    #     ["TRADE_SIZE", "bid_size_ex", "ask_size_ex"]
    # ]

    # classical
    mid_ex = 0.5 * (data["ask_ex"] + data["bid_ex"])
    mid_best = 0.5 * (data["BEST_ASK"] + data["BEST_BID"])
    x["rel_ask_ex"] = (data["TRADE_PRICE"] - mid_ex) / (data["ask_ex"] - mid_ex)
    x["rel_bid_ex"] = (mid_ex - data["TRADE_PRICE"]) / (mid_ex - data["bid_ex"])
    x["BEST_rel_bid"] = (data["TRADE_PRICE"] - mid_best) / (data["BEST_ASK"] - mid_best)
    x["BEST_rel_ask"] = (mid_best - data["TRADE_PRICE"]) / (mid_best - data["BEST_BID"])
    x["bid_ask_ratio_ex"] = data["bid_ex"] / data["ask_ex"]

    x["chg_ex_lead"] = data["TRADE_PRICE"] - data["price_ex_lead"]
    x["chg_ex_lag"] = data["TRADE_PRICE"] - data["price_ex_lag"]
    x["chg_all_lead"] = data["TRADE_PRICE"] - data["price_all_lead"]
    x["chg_all_lag"] = data["TRADE_PRICE"] - data["price_all_lag"]

    # log transformed features
    x[
         [
             "ask_ex",
             "bid_ex",
             "BEST_ASK",
             "BEST_BID",
             "TRADE_PRICE",
             "price_all_lag",
             "price_all_lead",
             "price_ex_lag",
             "price_ex_lead",
             "TRADE_SIZE", 
             "bid_size_ex", 
             "ask_size_ex",
         ]
     ] = np.log1p(data[
         [
             "ask_ex",
             "bid_ex",
             "BEST_ASK",
             "BEST_BID",
             "TRADE_PRICE",
             "price_all_lag",
             "price_all_lead",
             "price_ex_lag",
             "price_ex_lead",
             "TRADE_SIZE", 
             "bid_size_ex", 
             "ask_size_ex"
         ]
     ]
    )

    # impute with zeros
    x.replace([np.inf, -np.inf], np.nan, inplace=True)
    x.fillna(0, inplace=True)

    # scale to [-1, 1]
    if not hasattr(min_max_scaler, "n_features_in_"):
        min_max_scaler.fit(x)
    x[x.columns] = min_max_scaler.transform(x)

    # https://stackoverflow.com/questions/70727291/how-do-i-know-whether-a-sklearn-scaler-is-already-fitted-or-not

    # if not hasattr(oe_option_type, "n_features_in_"):
    #     oe_option_type.fit(data["OPTION_TYPE"].astype(str).values.reshape(-1, 1))
    # x["bin_option_type"] = oe_option_type.transform(
    #     data["OPTION_TYPE"].astype(str).values.reshape(-1, 1)
    # )

    # if not hasattr(oe_root, "n_features_in_"):
    #     oe_root.fit(data["ROOT"].astype(str).values.reshape(-1, 1))
    # x["bin_root"] = oe_root.transform(data["ROOT"].astype(str).values.reshape(-1, 1))

    # if not hasattr(oe_issue_type, "n_features_in_"):
    #     oe_issue_type.fit(data["issue_type"].astype(str).values.reshape(-1, 1))
    # x["bin_issue_type"] = oe_issue_type.transform(
    #     data["issue_type"].astype(str).values.reshape(-1, 1)
    # )

    # x.replace([np.inf, -np.inf], np.nan, inplace=True)

    x["buy_sell"] = data["buy_sell"]
    return x


In [8]:
output_path = (
    f"gs://thesis-bucket-option-trade-classification/data/{name}/train_set_60.parquet"
)
train = transform(train)
train.to_parquet(output_path)
dataset.add_reference(output_path)

del train

In [9]:
output_path = (
    f"gs://thesis-bucket-option-trade-classification/data/{name}/val_set_20.parquet"
)
val = transform(val)
val.to_parquet(output_path)
dataset.add_reference(output_path)

del val

In [10]:
output_path = (
    f"gs://thesis-bucket-option-trade-classification/data/{name}/test_set_20.parquet"
)
test = transform(test)
test.to_parquet(output_path)
dataset.add_reference(output_path)


[<ManifestEntry ref: gs://thesis-bucket-option-trade-classification/data/classical_size_features_log_normalized/test_set_20.parquet/test_set_20.parquet>]

In [11]:
test.describe()

,TRADE_PRICE,bid_ask_size_ratio_ex,rel_bid_size_ex,rel_ask_size_ex,rel_ask_ex,rel_bid_ex,BEST_rel_bid,BEST_rel_ask,bid_ask_ratio_ex,chg_ex_lead,...,BEST_ASK,BEST_BID,price_all_lag,price_all_lead,price_ex_lag,price_ex_lead,TRADE_SIZE,bid_size_ex,ask_size_ex,buy_sell
count,9.861576e+06,9.861576e+06,9.861576e+06,9.861576e+06,9.861576e+06,9.861576e+06,9.861576e+06,9.861576e+06,9.861576e+06,9.861576e+06,...,9.861576e+06,9.861576e+06,9.861576e+06,9.861576e+06,9.861576e+06,9.861576e+06,9.861576e+06,9.861576e+06,9.861576e+06,9.861576e+06
mean,-7.061342e-01,-9.998360e-01,-9.998290e-01,-9.998783e-01,-4.496103e-01,4.496103e-01,-2.217289e-01,2.217278e-01,-7.037069e-01,1.722710e-01,...,-8.189535e-01,-7.144768e-01,-7.731023e-01,-8.203375e-01,-7.321574e-01,-7.479942e-01,-8.128382e-01,-4.204213e-01,-3.656733e-01,-2.805495e-02
std,2.711683e-01,1.158564e-03,1.354047e-03,8.864031e-04,6.287551e-04,6.287550e-04,7.700371e-04,7.700391e-04,1.178656e-01,4.445568e-03,...,1.602128e-01,2.718154e-01,2.112202e-01,1.696631e-01,2.708560e-01,2.639858e-01,1.995284e-01,3.001729e-01,2.637453e-01,9.996064e-01
min,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-5.057560e-01,4.067200e-01,-3.200005e-01,-1.930445e-01,-1.000000e+00,-9.790342e-01,...,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00
25%,-9.195079e-01,-9.999950e-01,-9.999933e-01,-9.999936e-01,-4.499189e-01,4.493566e-01,-2.219640e-01,2.215066e-01,-7.333333e-01,1.721596e-01,...,-9.448681e-01,-9.294147e-01,-9.397130e-01,-9.544227e-01,-9.518099e-01,-9.648794e-01,-1.000000e+00,-5.834429e-01,-5.544227e-01,-1.000000e+00
50%,-7.777990e-01,-9.999808e-01,-9.999701e-01,-9.999773e-01,-4.495944e-01,4.495944e-01,-2.217396e-01,2.217385e-01,-6.525424e-01,1.722140e-01,...,-8.607004e-01,-7.878170e-01,-8.287887e-01,-8.662185e-01,-8.065964e-01,-8.251519e-01,-8.643808e-01,-4.034553e-01,-3.925944e-01,-1.000000e+00
75%,-5.676084e-01,-9.999556e-01,-9.998640e-01,-9.998918e-01,-4.493566e-01,4.499189e-01,-2.215077e-01,2.219629e-01,-6.220183e-01,1.723447e-01,...,-7.362892e-01,-5.769865e-01,-6.658070e-01,-7.341432e-01,-5.959212e-01,-6.147051e-01,-6.664538e-01,-2.222069e-01,-1.915243e-01,1.000000e+00
max,1.096489e+00,-8.631786e-01,-6.186268e-02,-3.209091e-01,-4.067197e-01,5.057546e-01,1.930428e-01,3.199995e-01,-1.000000e-01,7.770215e-01,...,2.405476e-01,1.096545e+00,5.909646e-01,2.682220e-01,1.029699e+00,1.000336e+00,9.747139e-01,6.547516e-01,6.051181e-01,1.000000e+00


In [12]:
run.log_artifact(dataset)
run.finish()
